In [1]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.keras.models import load_model
from keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, InputLayer, Dropout
from tensorflow.keras.models import Sequential
from tensorflow import keras
import os

In [3]:
model = keras.models.load_model('train/model')

In [13]:
df = pd.read_csv('train/frames.csv')
print("There are",df['Class'].nunique(), "Unique environments detected. Please enter their names")
val_dict = {}
count = 1
while count <= df['Class'].nunique():
    print("Please input the name of location", count)
    location = input()
    val_dict[count] = location
    count += 1
print(val_dict)

There are 4 Unique environments detected. Please enter their names
Please input the name of location 1


 check in


Please input the name of location 2


 lobby


Please input the name of location 3


 security


Please input the name of location 4


 waiting area


{1: 'check in', 2: 'lobby', 3: 'security', 4: 'waiting area'}


In [7]:
counter = int((len(os.listdir('userhalf')))/2)
user_dict = {}
for x in range(counter):
    calc_val = []
    test = pd.read_csv('userhalf/user%d/frames.csv' % x)
    test_image = []
    for img_name in test.Image_ID:
        img = plt.imread('userhalf/user%d/' % x + img_name)
        test_image.append(img)
    test_img = np.array(test_image)
    test_image = []
    for i in range(0,test_img.shape[0]):
        a = resize(test_img[i], preserve_range=True, output_shape=(224,224)).astype(int)
        test_image.append(a)
    test_image = np.array(test_image)
    print(test_image.shape)
    # preprocessing the images
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))  
    test_image = preprocess_input(test_image, mode='caffe')

    # extracting features from the images using pretrained model
    test_image = base_model.predict(test_image)

    # converting the images to 1-D form
    test_image = test_image.reshape(test_image.shape[0], 7*7*2048)

    # zero centered images
    test_image = test_image/test_image.max()
    user = model.predict_classes(test_image)
    for i in range(df['Class'].nunique()):
        calc_val.append(user[user==i+1].shape[0])
    user_dict[x] = calc_val
    print(user_dict[x])
print(user_dict)

(582, 224, 224, 3)
[88, 335, 69, 90]
(163, 224, 224, 3)
[25, 42, 83, 13]
(142, 224, 224, 3)
[3, 124, 12, 3]
(75, 224, 224, 3)
[10, 65, 0, 0]
(340, 224, 224, 3)
[125, 192, 23, 0]
(1065, 224, 224, 3)
[42, 932, 61, 30]
(127, 224, 224, 3)
[79, 0, 48, 0]
(1254, 224, 224, 3)
[85, 759, 410, 0]
(70, 224, 224, 3)
[3, 61, 6, 0]
(114, 224, 224, 3)
[74, 40, 0, 0]
(330, 224, 224, 3)
[96, 165, 31, 38]
(134, 224, 224, 3)
[67, 66, 1, 0]
(136, 224, 224, 3)
[94, 30, 9, 3]


KeyboardInterrupt: 

In [62]:
new_dict = {}
for i in val_dict:
    sums = []
    for j in user_dict:
        sums.append(user_dict[j][i-1])
    new_dict[i-1] = round(sum(sums)/df['Class'].nunique(), 2)
print(new_dict)

{0: 44.0, 1: 4.75, 2: 11.0, 3: 4.5}


In [8]:
print(user_dict)

{0: [88, 335, 69, 90], 1: [25, 42, 83, 13], 2: [3, 124, 12, 3], 3: [10, 65, 0, 0], 4: [125, 192, 23, 0], 5: [42, 932, 61, 30], 6: [79, 0, 48, 0], 7: [85, 759, 410, 0], 8: [3, 61, 6, 0], 9: [74, 40, 0, 0], 10: [96, 165, 31, 38], 11: [67, 66, 1, 0], 12: [94, 30, 9, 3]}


In [73]:
user_stdev = {}
for i in user_dict:
    stdev_calc = []
    for j in new_dict:
        stdev_calc.append(user_dict[i][j] - new_dict[j])
    new_calc = (math.sqrt((sum(stdev_calc) ** 2)/len(user_dict.keys())))
    user_stdev[i] = new_calc
print(user_stdev)

{0: 37.375, 1: 31.125, 2: 1.375, 3: 7.625}


In [116]:
fields = []
for i in val_dict:
    fields.append(val_dict[i])
print(fields)
rows = []
for i in user_dict:
    rows.append(('user' + str(i), user_dict[i]))
print(rows)

['check in', 'baggage', 'securityu', 'waiting']
[('user0', [131, 2, 6, 0]), ('user1', [2, 0, 0, 0]), ('user2', [34, 8, 23, 2]), ('user3', [9, 9, 15, 16])]


In [118]:
import csv
filename = "final_data.csv"
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows
    csvwriter.writerows(user_vals)

In [9]:
user_vals = []
test_vals = []
for i in user_dict:
    test_vals = []
    for j in new_dict:
        test_vals.append(user_dict[i][j])
    user_vals.append(test_vals)
    print(user_vals)

NameError: name 'new_dict' is not defined

In [110]:
my_df = pd.DataFrame(user_vals)
my_df.to_csv('my_csv.csv', index=False, header=False)

In [11]:
values = []
for i in user_dict:
    values.append(user_dict[i])
print(values)

[[88, 335, 69, 90], [25, 42, 83, 13], [3, 124, 12, 3], [10, 65, 0, 0], [125, 192, 23, 0], [42, 932, 61, 30], [79, 0, 48, 0], [85, 759, 410, 0], [3, 61, 6, 0], [74, 40, 0, 0], [96, 165, 31, 38], [67, 66, 1, 0], [94, 30, 9, 3]]


In [12]:
import csv
filename = "times.csv"
with open(filename, 'w',newline = '') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(values)

In [5]:
test = pd.read_csv('userhalf/user49/frames.csv')

In [6]:
test_image = []
for img_name in test.Image_ID:
    img = plt.imread('userhalf/user49/' + img_name)
    test_image.append(img)
test_img = np.array(test_image)

In [7]:
test_image = []
for i in range(0,test_img.shape[0]):
    a = resize(test_img[i], preserve_range=True, output_shape=(224,224)).astype(int)
    test_image.append(a)
test_image = np.array(test_image)
test_image.shape

(376, 224, 224, 3)

In [8]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) 
# preprocessing the images
test_image = preprocess_input(test_image, mode='caffe')

# extracting features from the images using pretrained model
test_image = base_model.predict(test_image)

# converting the images to 1-D form
test_image = test_image.reshape(376, 7*7*2048)

# zero centered images
test_image = test_image/test_image.max()

In [9]:
user1 = model.predict_classes(test_image)

C:\Users\lotla\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [10]:
print(user1)

[1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2
 2 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 2 2 2 2 2 2 2 2 2 2 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1
 1 1 1 1 3 3 1 3 2 2 2 2 2 3 2 2 3 3 3 3 3 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 2 1 3 3 3 3 3 3 1 2 1 1 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 4 4 4 4 4 4 4 4 4 4 1 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 1 4 4 4 1 4 4 4 4 4 4 4 4 4 4 1 1 4 1 1 4 4 4 1 1
 1 4 4 4 4 4]


In [11]:
probabilities = model.predict_proba(test_image)

C:\Users\lotla\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


In [12]:
print(probabilities)

[[1.42560306e-03 7.31980145e-01 1.14314295e-01 1.31523795e-02
  1.39127582e-01]
 [3.60249222e-04 2.52681389e-03 2.14175899e-02 9.48973835e-01
  2.67214570e-02]
 [3.36978381e-04 2.06013350e-03 1.64988954e-02 9.48629916e-01
  3.24740484e-02]
 ...
 [1.46428632e-04 1.26761617e-03 2.75981636e-03 9.81968820e-01
  1.38573525e-02]
 [3.39617894e-04 1.98050332e-03 7.05771670e-02 9.18889940e-01
  8.21277220e-03]
 [1.14085997e-04 1.39248348e-03 1.77755614e-03 9.90796685e-01
  5.91914728e-03]]


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2048)              205522944 
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 2

In [9]:
testprob = model.predict(test_image)

In [15]:
print(testprob[322])

[1.4920115e-04 3.0714658e-03 2.4883121e-03 6.4494587e-03 9.8784155e-01]


In [31]:
print(testprob[3])

[3.7243598e-04 2.3758614e-03 1.9271675e-02 9.4159120e-01 3.6388781e-02]


In [11]:
test_image2 = test_image/255

In [19]:
testprob = model.predict(test_image)

In [20]:
print(testprob)

[[2.4886669e-03 4.9353212e-01 2.7404994e-01 4.8320659e-02 1.8160862e-01]
 [1.1678940e-04 9.9191272e-01 3.9144875e-03 3.1526345e-03 9.0331165e-04]
 [1.1939214e-04 9.9170059e-01 3.9257728e-03 3.3166986e-03 9.3763776e-04]
 ...
 [1.7494624e-03 3.1831002e-01 2.7632771e-02 1.3880703e-01 5.1350075e-01]
 [1.7491698e-03 3.2151061e-01 2.7441060e-02 1.3898864e-01 5.1031047e-01]
 [1.7043854e-03 3.4847057e-01 2.5368460e-02 1.3031265e-01 4.9414399e-01]]


In [28]:
print(testprob[375])

[0.00170439 0.34847057 0.02536846 0.13031265 0.494144  ]


In [29]:
confidences = np.squeeze(model.predict(test_image))

In [30]:
print(confidences)

[[2.4886669e-03 4.9353212e-01 2.7404994e-01 4.8320659e-02 1.8160862e-01]
 [1.1678940e-04 9.9191272e-01 3.9144875e-03 3.1526345e-03 9.0331165e-04]
 [1.1939214e-04 9.9170059e-01 3.9257728e-03 3.3166986e-03 9.3763776e-04]
 ...
 [1.7494624e-03 3.1831002e-01 2.7632771e-02 1.3880703e-01 5.1350075e-01]
 [1.7491698e-03 3.2151061e-01 2.7441060e-02 1.3898864e-01 5.1031047e-01]
 [1.7043854e-03 3.4847057e-01 2.5368460e-02 1.3031265e-01 4.9414399e-01]]


In [16]:
print(confidences[9])

NameError: name 'confidences' is not defined

In [37]:
k = 5
confidences = np.squeeze(model.predict(test_image))
inds = np.argsort(-confidences)
top_k = inds[:k]
top_confidences = confidences[inds]

In [35]:
class_names = ['check in', 'baggage', 'securityu', 'waiting']

In [38]:
print(top_confidences)

[[[1.1678940e-04 9.9191272e-01 3.9144875e-03 3.1526345e-03 9.0331165e-04]
  [1.1939214e-04 9.9170059e-01 3.9257728e-03 3.3166986e-03 9.3763776e-04]
  [1.1143440e-04 9.9143010e-01 4.4988371e-03 3.3565226e-03 6.0316891e-04]
  [1.3404711e-04 9.9025553e-01 4.7455919e-03 3.8140684e-03 1.0506734e-03]
  [2.4886669e-03 4.9353212e-01 2.7404994e-01 4.8320659e-02 1.8160862e-01]]

 [[1.1678940e-04 9.9191272e-01 3.9144875e-03 3.1526345e-03 9.0331165e-04]
  [1.1939214e-04 9.9170059e-01 3.9257728e-03 3.3166986e-03 9.3763776e-04]
  [1.3404711e-04 9.9025553e-01 4.7455919e-03 3.8140684e-03 1.0506734e-03]
  [1.1143440e-04 9.9143010e-01 4.4988371e-03 3.3565226e-03 6.0316891e-04]
  [2.4886669e-03 4.9353212e-01 2.7404994e-01 4.8320659e-02 1.8160862e-01]]

 [[1.1678940e-04 9.9191272e-01 3.9144875e-03 3.1526345e-03 9.0331165e-04]
  [1.1939214e-04 9.9170059e-01 3.9257728e-03 3.3166986e-03 9.3763776e-04]
  [1.3404711e-04 9.9025553e-01 4.7455919e-03 3.8140684e-03 1.0506734e-03]
  [1.1143440e-04 9.9143010e-01 4.4